# Gaussian Process Regression on IrO3

# Import Modules

In [1]:
import numpy as np
import pandas as pd
from catlearn.regression import GaussianProcess
import matplotlib.pyplot as plt

# Read Data

In [2]:
train_x = (pd.read_csv('./RAW/train_x.csv', sep=',', header=None)).as_matrix()
train_y = (pd.read_csv('./RAW/train_y.csv', sep=',', header=None)).as_matrix()

test_data = (pd.read_csv('./RAW/test_features.csv', sep=',',
             header=None)).drop([0], axis=1).as_matrix()

/home/raulf2012/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/raulf2012/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/raulf2012/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


# Gaussian Process

In [3]:
# Define initial prediction parameters.
noise = 0.0042  # Regularisation parameter.
sigma_l = 6.3917  # Length scale parameter.
sigma_f = 0.5120  # Scaling parameter.
alpha = 0.3907  # Alpha parameter.

kdict = [
    {
        'type': 'quadratic',
        'dimension': 'single',
        'slope': sigma_l,
        'scaling': sigma_f,
        'degree':alpha
        }
    ]

gp = GaussianProcess(
    kernel_list=kdict, regularization=noise, train_fp=train_x,
    train_target=train_y, optimize_hyperparameters=True,
    scale_data=False)

print('Optimized kernel:', gp.kernel_list)

# Optimize hyperparameters:
# gp.optimize_hyperparameters(global_opt=False)

Optimized kernel: [{'type': 'quadratic', 'dimension': 'single', 'slope': array([0.50550311]), 'scaling': 0.47230283815705965, 'degree': array([0.15135033])}]


# Predictions

In [4]:
# Do the optimized predictions.
pred = gp.predict(test_fp=test_data[:249], uncertainty=True)
prediction = np.array(pred['prediction'][:, 0])

# Calculate the uncertainty of the predictions.
uncertainty = np.array(pred['uncertainty'])

# Sort predictions (ascending):
sorted_x = np.arange(0, len(prediction))
sorted_y = prediction[np.argsort(prediction)]
sorted_unc = uncertainty[np.argsort(prediction)]

# Plotting

In [5]:
import plotly.plotly as py
import plotly.graph_objs as go

import os

x_array = [0, 1, 2, 3]
y_array = [0, 1, 2, 3]


trace = go.Scatter(
    x=sorted_x,
    y=sorted_y,
    
    error_y={
        "type": 'data',
        "array": sorted_unc,
        "visible": True,
        },

    mode="markers",
    )

data = [trace]

py.iplot(data,
    filename=os.path.join(
        "__temp__",
        "temp_plot_0"
        )
    )

/home/raulf2012/miniconda2/envs/py36/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

